# Customer Profitability
-----------------------------------------------------------------------------

## ! Important
For this code I divided the data into several csv file. You need to have ALL the csv files to run the code.

In [1]:
import pandas as pd ; import numpy as np ; import itertools ; import matplotlib.pyplot as plt
import string

In [30]:
products = list(string.ascii_uppercase)[0:12]
products.remove('J')
products

col = ['Running Number', 'Customer id']
col += [i for i in products]
col

['Running Number',
 'Customer id',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'K',
 'L']

### Step 1 : Data and cleaning (be ready it's very long)
- First we take care of the direct cost file
- Second Sale ledge
- Third Indirect cost

<u> File 1 : Direct cost </u>

In [69]:
direct_cost = pd.read_csv('Direct cost sheet.csv', delimiter=';', encoding= 'unicode_escape')
direct_cost.dropna(axis = 0, how = 'all', inplace = True)
direct_cost.dropna(axis = 1, how = 'all', inplace = True)
direct_cost.drop(direct_cost.head(5).index,inplace=True)

# Create a dataframe for the components used in direct cost
direct_driver_used = direct_cost.iloc[:,:len(products)+2].copy()
direct_driver_used.columns = col
direct_driver_used = direct_driver_used.apply(lambda x: x.astype(np.int))
direct_driver_used.reset_index(drop=True, inplace = True)
direct_driver_used

,Running Number,Customer id,A,B,C,D,E,F,G,H,I,K,L
0,1,105,21,44,0,0,51,0,130,0,23,0,3
1,2,117,0,0,0,10,27,0,0,17,0,20,9
2,3,72,2,0,0,17,53,0,0,0,18,0,0
3,4,65,24,69,97,0,67,90,0,19,0,75,30
4,5,50,12,70,73,1,49,0,0,0,27,106,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2996,52,5,0,0,20,0,33,15,0,0,44,0
2996,2997,121,21,5,0,12,38,0,14,0,0,71,0
2997,2998,138,6,0,26,6,0,135,38,0,0,0,42
2998,2999,149,17,0,45,0,24,71,0,28,10,0,18


In [70]:
#Dataframe for direct cost per component per order
direct_cost_order_v1 = direct_cost.iloc[:,len(products)+2:len(products)*2+2].copy()
info = pd.DataFrame(direct_driver_used[['Running Number', 'Customer id']])
direct_cost_order = info.join(direct_cost_order_v1)
direct_cost_order.columns = col

# Clean the data by removing the euro, comma and - signs
direct_cost_order= direct_cost_order.replace('\x80','',regex=True)
direct_cost_order= direct_cost_order.replace(',','.',regex=True)
direct_cost_order= direct_cost_order.replace('-',0,regex=True)

# Convert all product columns to floats
direct_cost_order[products] = direct_cost_order[products].apply(pd.to_numeric)
direct_cost_order[direct_cost_order.select_dtypes(include = object).columns] = direct_cost_order.select_dtypes(include = object).apply(lambda x: x.astype(np.int))
direct_cost_order.reset_index(drop=True, inplace = True)
direct_cost_order

,Running Number,Customer id,A,B,C,D,E,F,G,H,I,K,L
0,1,105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2996,52,6.80,4.52,9.32,0.00,0.00,0.0,18.05,0.00,7.67,6.36,7.44
2996,2997,121,13.64,0.00,17.67,0.00,5.19,0.0,8.21,0.00,0.00,6.57,0.00
2997,2998,138,0.00,3.88,0.00,0.00,0.00,0.0,0.00,0.00,6.25,16.34,9.33
2998,2999,149,13.78,0.00,0.00,0.00,0.00,0.0,0.00,9.82,6.38,0.00,11.49


<u> File 2: Sales ledger </u>

In [71]:
sales_ledger = pd.read_csv('Sales ledger.csv', delimiter=';', encoding= 'unicode_escape')
sales_ledger.dropna(axis = 0, how = 'all', inplace = True)
sales_ledger.dropna(axis = 1, how = 'all', inplace = True)
sales_ledger.drop(sales_ledger.tail(1).index,inplace=True)
sales_ledger.drop(sales_ledger.head(2).index,inplace=True)
sales_ledger.reset_index(drop=True, inplace = True)
sales_ledger

,Unnamed: 0,Unnamed: 1,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
0,1,105,17,0,63,0,28,0,95,0,..., -," 49,00", -," 16,00", -," 6,00", -, -," 17,00"," 18,00"
1,2,117,16,0,0,17,0,89,0,11,..., -, -," 5,00", -," 103,00", -," 18,00"," 15,00", -, -
2,3,72,30,31,0,0,25,63,74,55,...," 9,00", -, -," 15,00"," 167,00"," 14,00"," 15,00"," 16,00", -," 20,00"
3,4,65,4,0,24,0,44,92,0,58,..., -," 49,00", -," 19,00"," 101,00", -," 17,00", -, -, -
4,5,50,18,0,0,2,0,0,43,0,..., -, -," 17,00", -, -," 30,00", -," 19,00"," 16,00", -
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2996,52,4,9,0,0,0,43,0,0,...," 7,00", -, -, -," 139,00", -, -," 20,00"," 16,00", -
2996,2997,121,20,28,72,0,0,0,28,0,...," 8,00"," 20,00", -, -, -," 36,00", -, -," 18,00", -
2997,2998,138,0,41,0,0,10,0,82,0,...," 10,00", -, -," 15,00", -," 22,00", -, -," 19,00", -
2998,2999,149,11,30,38,0,0,0,11,21,...," 6,00"," 33,00", -, -, -," 20,00"," 16,00"," 15,00", -," 18,00"


In [72]:
# Create a dataframe for the units sold
sale_product = sales_ledger.iloc[:,:len(products)+2].copy()
sale_product.columns = col
sale_product = sale_product.apply(lambda x: x.astype(np.int))
sale_product.reset_index(drop=True, inplace = True)
sale_product

,Running Number,Customer id,A,B,C,D,E,F,G,H,I,K,L
0,1,105,17,0,63,0,28,0,95,0,0,90,34
1,2,117,16,0,0,17,0,89,0,11,3,0,0
2,3,72,30,31,0,0,25,63,74,55,16,0,35
3,4,65,4,0,24,0,44,92,0,58,0,0,0
4,5,50,18,0,0,2,0,0,43,0,17,66,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2996,52,4,9,0,0,0,43,0,0,13,101,0
2996,2997,121,20,28,72,0,0,0,28,0,0,36,0
2997,2998,138,0,41,0,0,10,0,82,0,0,13,0
2998,2999,149,11,30,38,0,0,0,11,21,8,0,30


In [73]:
#Dataframe for direct cost per component per order
sale_price_unit_v1 = sales_ledger.iloc[:,len(products)+2:len(products)*2+2].copy()
info = pd.DataFrame(sale_product[['Running Number', 'Customer id']])
sale_price_unit = info.join(sale_price_unit_v1)
sale_price_unit.columns = col

# Clean the data by removing the euro, comma and - signs
sale_price_unit= sale_price_unit.replace('\x80','',regex=True)
sale_price_unit= sale_price_unit.replace(',','.',regex=True)
sale_price_unit= sale_price_unit.replace('-',0,regex=True)

# Convert all product columns to floats
sale_price_unit[products] = sale_price_unit[products].apply(pd.to_numeric)
sale_price_unit[sale_price_unit.select_dtypes(include = object).columns] = sale_price_unit.select_dtypes(include = object).apply(lambda x: x.astype(np.int))
sale_price_unit.reset_index(drop=True, inplace = True)
sale_price_unit


,Running Number,Customer id,A,B,C,D,E,F,G,H,I,K,L
0,1,105,327.0,16.0,0.0,49.0,0.0,16.0,0.0,6.0,0.0,0.0,17.0
1,2,117,136.0,19.0,0.0,0.0,5.0,0.0,103.0,0.0,18.0,15.0,0.0
2,3,72,329.0,18.0,9.0,0.0,0.0,15.0,167.0,14.0,15.0,16.0,0.0
3,4,65,222.0,18.0,0.0,49.0,0.0,19.0,101.0,0.0,17.0,0.0,0.0
4,5,50,146.0,19.0,0.0,0.0,17.0,0.0,0.0,30.0,0.0,19.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2996,52,170.0,18.0,7.0,0.0,0.0,0.0,139.0,0.0,0.0,20.0,16.0
2996,2997,121,184.0,15.0,8.0,20.0,0.0,0.0,0.0,36.0,0.0,0.0,18.0
2997,2998,138,146.0,0.0,10.0,0.0,0.0,15.0,0.0,22.0,0.0,0.0,19.0
2998,2999,149,149.0,18.0,6.0,33.0,0.0,0.0,0.0,20.0,16.0,15.0,0.0


<u> File 3 :Indirect cost </u>

In [74]:
indirect_costs= pd.read_csv('Indirect cost.csv', delimiter=';', encoding= 'unicode_escape')
indirect_costs.dropna(axis = 0, how = 'all', inplace = True)
indirect_costs.dropna(axis = 1, how = 'all', inplace = True)
# indirect_costs.drop(indirect_costs.tail(1).index,inplace=True)
# indirect_costs.drop(indirect_costs.head(2).index,inplace=True)
indirect_costs.reset_index(drop=True, inplace = True)
indirect_costs


,Unnamed: 0,Activity cost,Cost driver,Explanation,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,Sales,750000.0,Number order lines,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Order incoming from customers,650000.0,Number order lines,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Number components per pallet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Warehouse,467000.0,Number pallets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Production/Assembly,1700000.0,Number components,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Quality control,450000.0,Number components,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Warehouse out,340000.0,Number sold units,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Customer support,250000.0,Number products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,Each component comes on a pallet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,One pallet contains the following amount of co...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


This file is in 2 parts the first 11 lines are the description of the activities, their activity cost and their cost driver. <br>
The secon part is the detail of how many pallets are used per products. <br>
I will create 2 dataframes, one for the activities costs and the other of the warehouse drivers. 

In [75]:
cost_info = indirect_costs.iloc[:8,:3].copy()
cost_info = cost_info.rename({'Unnamed: 0':'Activity'}, axis=1)
cost_info.dropna(axis = 0, how = 'any', inplace = True)
cost_info.reset_index(drop=True, inplace = True)
cost_info

,Activity,Activity cost,Cost driver
0,Sales,750000.0,Number order lines
1,Order incoming from customers,650000.0,Number order lines
2,Warehouse,467000.0,Number pallets
3,Production/Assembly,1700000.0,Number components
4,Quality control,450000.0,Number components
5,Warehouse out,340000.0,Number sold units
6,Customer support,250000.0,Number products


In [76]:
components_pallet = indirect_costs.iloc[-1,3:].copy()
components_pallet = pd.DataFrame(components_pallet).T
components_pallet.columns = products
components_pallet.reset_index(drop=True, inplace = True)
components_pallet

,A,B,C,D,E,F,G,H,I,K,L
0,405,339,102,498,236,475,371,398,138,385,322
